<a href="https://colab.research.google.com/github/thehimalayanleo/Private-Machine-Learning/blob/master/FederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft
import torch as th
import syft as sy

     |████████████████████████████████| 317kB 2.9MB/s 
     |████████████████████████████████| 389kB 44.2MB/s 
     |████████████████████████████████| 460kB 38.9MB/s 
     |████████████████████████████████| 204kB 39.3MB/s 
     |████████████████████████████████| 2.1MB 24.0MB/s 
     |████████████████████████████████| 256kB 39.5MB/s 
     |████████████████████████████████| 81kB 25.5MB/s 
     |████████████████████████████████| 51kB 16.4MB/s 
     |████████████████████████████████| 266kB 42.0MB/s 
     |████████████████████████████████| 122kB 39.8MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076279 sha256=a1c7c669e8c08322e50f414e6100a8765e9f58da061b6f7e078f75bf8032d28c
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=6d3fecd5b4543f660ac0d9dfed030c6927ff2a09339b06199d89213d6bc275dc
  Stored in di

In [0]:
hook = sy.TorchHook(th)

In [3]:
bob = sy.VirtualWorker(hook, id='bob')
bob._objects

{}

In [13]:
data = th.tensor([1, 2, 3, 4, 5])
data = data.send(bob)
bob._objects

{59691190825: tensor([1, 2, 3, 4, 5]),
 71418800096: (Wrapper)>[MultiPointerTensor]
 	-> tensor([1, 2, 3, 4, 5])
 	-> [PointerTensor | bob:9583878115 -> alice:75161749740],
 91727570157: tensor([1, 2, 3, 4, 5])}

In [14]:
data.location
print(data.location == bob)
print(data.id_at_location, data.id, data.owner)

True
59691190825 7098720106 <VirtualWorker id:me #objects:0>


In [15]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [16]:
data = data.get()
data

tensor([1, 2, 3, 4, 5])

In [17]:
bob._objects

{71418800096: (Wrapper)>[MultiPointerTensor]
 	-> tensor([1, 2, 3, 4, 5])
 	-> [PointerTensor | bob:9583878115 -> alice:75161749740],
 91727570157: tensor([1, 2, 3, 4, 5])}

In [31]:
alice = sy.VirtualWorker(hook, id='alice')
new_data = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)
new_data

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:80292355118 -> bob:44521179710]
	-> [PointerTensor | me:87424280540 -> alice:21641016336]

In [33]:
new_data.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [0]:
x = th.tensor([1, 2, 3]).send(bob)
y = th.tensor([1, 1, 1]).send(bob)

In [37]:
z = x + y
z.get()

tensor([2, 3, 4])

In [0]:
x = th.tensor([1., 2, 3], requires_grad=True).send(bob)
y = th.tensor([1., 1, 1], requires_grad=True).send(bob)

In [41]:
z = (x+y).sum() 
z.backward()

(Wrapper)>[PointerTensor | me:38283006455 -> bob:36902701303]

In [44]:
x = x.get()
x.grad

tensor([1., 1., 1.])

In [0]:
## Learn a simple linear model

input = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
targets = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)
weights = th.tensor([[0.], [0]], requires_grad=True).send(bob)

In [59]:
loss1 = 10
rounds=0
while rounds < 30 or loss1 > 1e-5:

  pred = th.mm(input, weights)
  loss = ((pred-targets)**2).sum()

  loss.backward()  

  weights.data.sub_(weights.grad*0.1)
  weights.grad*=0
  loss1 = loss.get().data
  print(loss1, rounds)
  rounds+=1



tensor(2.) 0
tensor(0.5600) 1
tensor(0.2432) 2
tensor(0.1372) 3
tensor(0.0849) 4
tensor(0.0538) 5
tensor(0.0344) 6
tensor(0.0220) 7
tensor(0.0141) 8
tensor(0.0090) 9
tensor(0.0058) 10
tensor(0.0037) 11
tensor(0.0024) 12
tensor(0.0015) 13
tensor(0.0010) 14
tensor(0.0006) 15
tensor(0.0004) 16
tensor(0.0003) 17
tensor(0.0002) 18
tensor(0.0001) 19
tensor(6.6461e-05) 20
tensor(4.2535e-05) 21
tensor(2.7223e-05) 22
tensor(1.7423e-05) 23
tensor(1.1150e-05) 24
tensor(7.1362e-06) 25
tensor(4.5672e-06) 26
tensor(2.9229e-06) 27
tensor(1.8707e-06) 28
tensor(1.1973e-06) 29


In [63]:
bob.clear_objects()
bob._objects

{}

In [67]:
x = th.tensor([1., 1]).send(bob)
print(bob._objects, x.child.garbage_collect_data)
del x
print(bob._objects)

{11470161431: tensor([1., 1.])} True
{}


In [68]:
for indx in range(1000):
  x = th.tensor([1, 1]).send(bob)
bob._objects

{49952626889: tensor([1, 1])}

In [0]:
## Learning a toy FL on two different machines

from torch import nn, optim

input = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True)
targets = th.tensor([[1.], [1], [0], [0]], requires_grad=True)
weights = th.tensor([[0.], [0]], requires_grad=True)

model = nn.Linear(2, 1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
for 